In [2]:
import cv2
import PIL
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import Video, widgets
from IPython.display import display

%matplotlib inline

In [3]:
video_path = 'Chess_Data\\chess_easy1.mp4'

In [4]:
video_capture = cv2.VideoCapture(video_path)

In [5]:
def save_first_frame(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Read the first frame
    ret, frame = cap.read()

    # Check if the frame was read successfully
    if not ret:
        print("Error: Could not read the first frame.")
        return

    # Release the video capture object
    cap.release()

    return frame

first_frame = save_first_frame(video_path)

In [6]:
def find_closest_points_to_corners(points, corners):
    closest_points = []
    for corner in corners:
        distances = np.linalg.norm(points - corner, axis=1)
        closest_point_index = np.argmin(distances)
        closest_points.append(points[closest_point_index])
    return np.array(closest_points)

In [7]:
gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edges = cv2.Canny(gray, 50, 150)

frame_with_boxes = first_frame.copy()
dst = cv2.cornerHarris(blurred, 5, 3, 0.02)
dst = cv2.dilate(dst, None)
corners = np.argwhere(dst > 0.01 * dst.max())

# Find corners closest to image corners
image_corners = np.array([[0, 0], [0, first_frame.shape[1]], [first_frame.shape[0], 0], [first_frame.shape[0], first_frame.shape[1]]])
closest_corners = find_closest_points_to_corners(corners, image_corners)

target_square_size = 1000  # Adjust the size as needed
target_points = np.array([[0, 0], [0, target_square_size - 1], [target_square_size - 1, 0], [target_square_size - 1, target_square_size - 1]], dtype=np.float32)

In [8]:
if "google.colab" in str(get_ipython()):
    from google.colab.patches import cv2_imshow

    imshow = cv2_imshow
else:

    def imshow(img):
        img = img.clip(0, 255).astype("uint8")
        if img.ndim == 3:
            if img.shape[2] == 4:
                img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGBA)
            else:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        display(PIL.Image.fromarray(img))

In [9]:
homography_matrix, _ = cv2.findHomography(closest_corners[:,::-1], target_points)
calibrated_image = cv2.warpPerspective(first_frame, homography_matrix, (target_square_size, target_square_size))
# imshow(calibrated_image)

In [10]:
def find_closest_points_to_corners(points, corners):
    closest_points = []
    for corner in corners:
        distances = np.linalg.norm(points - corner, axis=1)
        closest_point_index = np.argmin(distances)
        closest_points.append(points[closest_point_index])
    return np.array(closest_points)

def get_harris(chessboard):
    gray = cv2.cvtColor(chessboard, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    dst = cv2.cornerHarris(blurred, 5, 3, 0.02)
    dst = cv2.dilate(dst, None)
    corners = np.argwhere(dst > 0.01 * dst.max())
    return corners, chessboard.copy()

def get_closest_corners(corners, chessboard):
    image_corners = np.array([[0, 0], 
                              [0, chessboard.shape[1]], 
                              [chessboard.shape[0], 0], 
                              [chessboard.shape[0], chessboard.shape[1]]])
    closest_corners = find_closest_points_to_corners(corners, image_corners)
    return closest_corners

In [12]:
def detect_hand(frame):
    img_hls = cv2.cvtColor(frame, cv2.COLOR_BGR2HLS)

    # Define the skin color range in HLS
    skin_color_lower = np.array([0, 190, 100], dtype=np.uint8)
    skin_color_upper = np.array([15, 240, 255], dtype=np.uint8)

    # Create a mask using the skin color range
    range_mask = cv2.inRange(img_hls, skin_color_lower, skin_color_upper)

    # Remove noise with blurring
    blurred = cv2.blur(range_mask, (10, 10))

    # Threshold the blurred image to create a binary mask
    _, thresholded = cv2.threshold(blurred, 200, 255, cv2.THRESH_BINARY)

    roi1 = thresholded[0:img_hls.shape[0], 0:img_hls.shape[1]//2]
    roi2 = thresholded[0:int(img_hls.shape[0]/1.7), img_hls.shape[1]//2:img_hls.shape[1]]

    if np.any(roi1 == 255) or np.any(roi2 == 255):
        return True
    
    return False

In [11]:
video_capture = cv2.VideoCapture('Chess_Data\\chess_hard1.mp4')

font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 2
thickness = 2

i = 0
while True:
    i += 1
    ret, frame = video_capture.read()

    if not ret:
        break
    
    if detect_hand(frame):
        cv2.putText(frame, f"Hand detected!", (20, 100), font, font_scale, (0, 255, 0), thickness)

    # Display the frame with hand detection
    cv2.imshow('Hand Detection', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

In [34]:
# video_capture = cv2.VideoCapture('Chess_Data\\chess_hard1.mp4')

# font = cv2.FONT_HERSHEY_SIMPLEX
# font_scale = 2
# thickness = 2

# ret, prev_frame = video_capture.read()
# prev_gray = cv2.cvtColor(calibrated_image, cv2.COLOR_BGR2GRAY)
# ECHO_LENGTH = 40
# COUNTOUR_THRESHOLD = 120
# AREA_THRESHOLD = 30
# echo = False
# curr_echo = 0
# target_square_size = 1000
# i = 0
# while True:
#     i += 1   
#     hand = False
#     ret, chessboard = video_capture.read()
#     if not ret:
#         break

#     corners, frame_with_boxes = get_harris(chessboard)
#     closest_corners = get_closest_corners(corners, chessboard)
#     target_points = np.array([[0, 0], 
#                               [0, target_square_size - 1], 
#                               [target_square_size - 1, 0], 
#                               [target_square_size - 1, target_square_size - 1]], dtype=np.float32)
    
#     homography_matrix, _ = cv2.findHomography(closest_corners[:,::-1], target_points)
#     calibrated_image = cv2.warpPerspective(chessboard, homography_matrix, (target_square_size, target_square_size))
#     offset = 60
#     clean_img = calibrated_image.copy()

#     for x in np.linspace(offset, target_square_size-offset, 9, dtype=int):
#         cv2.line(clean_img, (x, 0+offset), (x, target_square_size-offset), color=(0, 0, 255), thickness=2)  
#         cv2.line(clean_img, (0+offset, x), (target_square_size-offset, x), color=(0, 0, 255), thickness=2)
  
#     gray = cv2.cvtColor(calibrated_image, cv2.COLOR_BGR2GRAY)
#     frame_diff = cv2.absdiff(prev_gray, gray)
#     _, thresholded_diff = cv2.threshold(frame_diff, 30, 255, cv2.THRESH_BINARY)
#     contours, _ = cv2.findContours(thresholded_diff, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#     num_contours = sum(1 for contour in contours if cv2.contourArea(contour) > AREA_THRESHOLD)
#     if num_contours > COUNTOUR_THRESHOLD or echo == True: # hand detected
#         hand = True
        
#         # chessboard = prev_frame
#         # curr_echo+=1
#         # if curr_echo>ECHO_LENGTH:
#         #     curr_echo = 0
#         #     echo = False
        
#     else: # hand not detected
#         prev_frame = chessboard
#         # echo = True

#     if hand:
#         cv2.putText(clean_img, f"Hand detected!", (20, 100), font, font_scale, (0, 255, 0), thickness)
#     prev_gray = gray.copy()
    
#     cv2.imshow('Frame with Giant Bounding Box', clean_img)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# video_capture.release()
# cv2.destroyAllWindows()

In [29]:
# video_capture = cv2.VideoCapture('Chess_Data\\chess_hard1.mp4')
# video_capture2 = cv2.VideoCapture('Chess_Data\\chess_hard1.mp4')


# i = 0
# while True:
#     i += 1
#     ret, frame = video_capture.read()
#     frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     _, hand_mask = cv2.threshold(frame_gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

#     if not ret:
#         break
    
#     if i > 60:
#         ret2, frame2 = video_capture.read()
#         frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
#         _, hand_mask2 = cv2.threshold(frame2_gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
#     else:
#         frame2 = np.zeros_like(frame)
#         hand_mask2 = np.zeros_like(hand_mask)


#     # Display the frame with hand detection
#     cv2.imshow('Hand Detection', hand_mask)
    
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break


# video_capture.release()
# cv2.destroyAllWindows()